In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
import requests
import folium

### Creamos una función para unir todo los Datasets y agregamos columnas faltantes.

Los datos dentro de los csv's no seguian un patrón adecuado en algunas de sus filas (no se encontraban estructurados como cuadro).

- Para adaptarlos a un DataFrame se extrajeron datos relevantes como ``ciudad_origen`` `estado_origen` `aeropuerto_origen` y `codigo_aeropuerto_origen`.
    -   Fue necesario para extraer los datos el uso de la libreria **re**. 


- Creamos un Datraframe con el que trabajar

In [5]:
# Función para leer y unir los datasets.
def unir_datasets(ruta):
    dfs = []

    for nombre_archivo in os.listdir(ruta):

        if nombre_archivo.endswith('.csv'):

            ruta_archivo = os.path.join(ruta, nombre_archivo)
            df = pd.read_csv(ruta_archivo, skiprows = 7, skipfooter = 1, engine = 'python') # Eliminamos las primeras filas y la última de cada dataset.

            with open(ruta_archivo, 'r') as archivo:
                segunda_fila = archivo.readlines()[1].strip() # Rescatamos información de la segunda fila de los csv's.
                coincidencia = re.search(r'Origin Airport: (.+), (.+): (.+) \((.+)\)', segunda_fila)

                if coincidencia:

                    ciudad_origen                  = coincidencia.group(1)
                    estado_origen                  = coincidencia.group(2)
                    aeropuerto_origen              = coincidencia.group(3)
                    codigo_aeropuerto_origen       = coincidencia.group(4)

                    df['ciudad_origen']            = ciudad_origen
                    df['estado_origen']            = estado_origen
                    df['aeropuerto_origen']        = aeropuerto_origen
                    df['codigo_aeropuerto_origen'] = codigo_aeropuerto_origen

                    dfs.append(df)

                else:
                    print("La segunda fila no tiene el formato esperado en el archivo:", nombre_archivo)

    resultado_df = pd.concat(dfs, ignore_index=True)
    resultado_df.to_pickle('data/pickle/vuelos.pkl')


ruta = r'data' # Ruta en la que se encuentren todos los csv's.
unir_datasets(ruta)

In [7]:
df = pd.read_pickle("data/pickle/vuelos.pkl")

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.head()

## Columnas del Dataframe


- **aerolinea**: Nombre de la aerolínea.
- **fecha**: La fecha del vuelo en formato día/mes/año.
- **numero_vuelo**: El número de vuelo.
- **numero_cola**: El número de cola del vuelo.
- **aeropuerto_destino**: El aeropuerto de destino del vuelo. 
- **hora_salida_programada**: La hora programada de salida del vuelo.
- **hora_salida_real**: La hora real de salida del vuelo.
- **duracion_programada_vuelo**: La duración programada del vuelo.
- **duracion_real**: La duración real del vuelo.
- **retraso_salida**: El retraso en la salida del vuelo.
- **hora_despegue**: La hora de despegue del vuelo.
- **tiempo_pista_salida**: El tiempo en pista antes del despegue.
- **tiempo_retraso_aerolinea**: Tiempo de retraso atribuido a la aerolínea.
- **tiempo_retraso_clima**: Tiempo de retraso atribuido al clima.
- **tiempo_retraso_sistema_aviacion**: Tiempo de retraso atribuido al sistema de aviación.
- **tiempo_retraso_seguridad**: Tiempo de retraso atribuido a cuestiones de seguridad.
- **retraso_llegada_aeronave**: Retraso en la llegada de la aeronave.
- **ciudad_origen**: La ciudad de origen del vuelo.
- **estado_origen**: El estado de origen del vuelo.
- **aeropuerto_origen**: El aeropuerto de origen del vuelo.
- **dia_semana**: El día de la semana en que tuvo lugar el vuelo.
- **año**: El año en que tuvo lugar el vuelo.
- **fin_de_semana**: Indicador de si el vuelo tuvo lugar durante el fin de semana o no.
- **festivos**: Indicador de si el vuelo tuvo lugar en un día festivo o no.
- **distancia_millas**: La distancia del vuelo en millas.
- **ciudad_destino**: La ciudad de destino del vuelo.
- **estado_destino**: El estado de destino del vuelo.

Aplicamos un diccionario de nuestra propia cosecha. Gracias a este diccionario, fuimos capaces de comprender mejor las columnas, además de poder manejarlas con mayor fluidez.

In [11]:
# RENOMBRAMOS LAS COLUMNAS PARA QUE SEAN MÁS EXPLICATIVAS 

df.rename(columns={ 
    'Carrier Code'                            : 'aerolinea',
    'Date (MM/DD/YYYY)'                       : 'fecha',
    'Flight Number'                           : 'numero_vuelo',
    'Tail Number'                             : 'numero_cola',
    'Destination Airport'                     : 'codigo_aeropuerto_destino',
    'Scheduled departure time'                : 'hora_salida_programada',
    'Actual departure time'                   : 'hora_salida_real',
    'Scheduled elapsed time (Minutes)'        : 'duracion_programada_vuelo',
    'Actual elapsed time (Minutes)'           : 'duracion_real',
    'Departure delay (Minutes)'               : 'retraso_salida',
    'Wheels-off time'                         : 'hora_despegue',
    'Taxi-Out time (Minutes)'                 : 'tiempo_pista_salida',
    'Delay Carrier (Minutes)'                 : 'tiempo_retraso_aerolinea',
    'Delay Weather (Minutes)'                 : 'tiempo_retraso_clima',
    'Delay National Aviation System (Minutes)': 'tiempo_retraso_sistema_aviacion',
    'Delay Security (Minutes)'                : 'tiempo_retraso_seguridad',
    'Delay Late Aircraft Arrival (Minutes)'   : 'retraso_llegada'
}, inplace= True)

Utilizando los datos de la **URL** https://www.transtats.bts.gov/ONTIME/CarrierInfo.html, traducimos los códigos de aeropuertos para mejor comprensión de los datos.

In [12]:
# RENOMBRAMOS LOS CÓDIGOS DE LAS AEROLÍNEAS

aerolineas = {
    'MQ': 'Envoy Air',
    'OO': 'SkyWest Airlines',
    'OH': 'PSA Airlines',
    '9E': 'Endeavor Air',
    'YV': 'Mesa Airlines',
    'WN': 'Southwest Airlines',
    'G4': 'Allegiant Air',
    'AA': 'American Airlines',
    'DL': 'Delta Air Lines',
    'F9': 'Frontier Airlines',
    'B6': 'JetBlue Airways',
    'YX': 'Republic Airways',
    'UA': 'United Airlines',
    'AS': 'Alaska Airlines',
    'HA': 'Hawaiian Airlines',
    'QX': 'Horizon Air',
    'NK': 'Spirit Airlines'
}

df['aerolinea'] = df['aerolinea'].replace(aerolineas)

### Buscamos y eliminamos duplicados.

In [ ]:
duplicados = df.duplicated()

if duplicados.any():
    print("El DataFrame tiene filas duplicadas.")
else:
    print("El DataFrame no tiene filas duplicadas.")

num_filas_duplicadas = df.duplicated().sum()

print("Número de filas duplicadas:", num_filas_duplicadas)

In [ ]:
df.shape

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

Hacemos una visualización de los valores Nan y un cálculo de los mismos.

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(df.isnull(), cmap='viridis', cbar=False)
plt.title('Visualización de NaNs')
plt.xlabel('Columnas')
plt.ylabel('Filas')
plt.show()

In [ ]:
# Calcular el porcentaje de Nan's que existe por columnas.
df.isna().sum()/len(df)*100

Observamos la prensencia de Nan's en la columna `numero_cola`

Existe relación con los números de cola con algunos número de vuelo, creamos una función para buscar el número de cola 

asociado al número de vuelo de esa fila en el diccionario y reemplazamos los NaN.

In [18]:
def rellenar_numero_cola_relacionado(df):

    vuelo_a_cola = dict(zip(df['numero_vuelo'], df['numero_cola']))

    def rellenar(fila):

        if pd.isnull(fila['numero_cola']):

            return vuelo_a_cola.get(fila['numero_vuelo'], pd.NA)
        
        else:

            return fila['numero_cola']

    df['numero_cola'] = df.apply(rellenar, axis = 1)

    return df

df_prueba = rellenar_numero_cola_relacionado(df)

In [ ]:
# Contamos el número de filas sin rellenar.
nan_cola = df['numero_cola'].isna().sum()

print(f'Quedan sin rellenar {nan_cola} filas en la columna `numero_cola`.')

# Las que no se rellenan las eliminamos. 
df.dropna(subset=['numero_cola'], inplace=True)

`numero_cola`: Número de cola de la aeronave.

La columna `numero_cola` contiene números de registro, son asignados por la Administración Federal de Aviación (FAA) en los Estados Unidos y se utilizan para identificar de manera única a cada aeronave registrada en EEUU.

- La letra inicial 'N' indica que la aeronave está registrada en EEUU.
- La letra inicial 'S' indica que se trata de una aeronave ultraligera.

Luego, los números y las letras restantes forman un identificador único para esa aeronave específica.

In [ ]:
# Comprobamos si todos son de origen estadounidense.
df['numero_cola_inicio'] = df['numero_cola'].apply(lambda x: x[0] if isinstance(x, str) else None)

print(df['numero_cola_inicio'].unique())

df.drop(columns = 'numero_cola_inicio', axis = 1, inplace = True)

Vemos que no todos los vuelos tiene N en el número de cola, así que no todos son de origen estadounidense.

### Agregamos y modificamos las columnas referentes a fechas y horas.

Utilizando la libreria **datetime** convertimos a su formato idóneo (*datetime* para las fechas y *time* para las horas) de las siguientes columnas:

- `hora_salida_real`, `hora_salida_programada`, `hora_llegada_real`, `fecha`, `dia_semana`, `anio`, `fin_de_semana`, `festivos`.

In [21]:
# Cambiamos a DATETIME las columnas 'hora_salida_real' y 'hora_salida_programada'.
df['hora_salida_real'] = df['hora_salida_real'].replace({'24:00' : '00:00'})

df['hora_salida_real'] = pd.to_datetime(df['hora_salida_real'], format = '%H:%M').dt.time
df['hora_salida_programada'] = pd.to_datetime(df['hora_salida_programada'], format='%H:%M').dt.time

In [22]:
# Sumamos la duración real al tiempo de salida real para obtener la hora de llegada.

df['hora_llegada_real'] = pd.to_timedelta(df['hora_salida_real'].astype(str)) + pd.to_timedelta(df['duracion_real'], unit='m')

In [ ]:
# Volvemos a convertir la columna 'hora_llegada_real a datetime.
df['hora_llegada_real'] = df['hora_llegada_real'].astype(str)

# Si deseas eliminar los microsegundos, puedes hacerlo así
df['hora_llegada_real'] = df['hora_llegada_real'].str.split('.').str[0]
df['hora_llegada_real'] = df['hora_llegada_real'].str.split(' ').str[2]

df['hora_llegada_real'] = pd.to_datetime(df['hora_llegada_real']).dt.time

In [24]:
# Cambiamos a DATETIME la columna 'fecha'.
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce')

# Agregamos las columnas año, día de la semana, mes y fin de semana.

df['dia_semana'] = df['fecha'].dt.dayofweek
df['anio'] = df['fecha'].dt.year
df['mes'] = df['fecha'].dt.month
df['fin_de_semana'] = (df['fecha'].dt.dayofweek >= 5).astype(int)

In [ ]:
# Agregamos el pkl con los días festivos, quitando las columnas innecesarias.
df_festivos = pd.read_pickle(r"data/fecha_festivos.pkl")

eliminar_columnas = ['dia', 'mes', 'ano']
df_festivos.drop(eliminar_columnas, axis = 1, inplace = True)

df_festivos.tail()

In [26]:
# Agregamos al df los días festivos de EEUU.
df = df.merge(df_festivos, left_on = 'fecha', right_on = 'festivos', how = 'left')

df['festivos'] = df['festivos'].apply(lambda x: 1 if pd.notnull(x) else 0)

In [ ]:
# Comprobamos que la columna se agrega bien.
prueba_fecha = df[df['fecha'] == '2023-12-25'] # Fecha festiva (Navidad).
prueba_fecha.head()

### Agregamos columnas referentes a los nombres de los aeropuertos de origen y destino.

- `codigo_aeropuerto_origen`, `codigo_aeropuerto_destino`.

In [ ]:
# Los códigos de aeropuerto de origen y destino coinciden menos en uno.
lista_codigo_aeropuerto_destino = sorted(df["codigo_aeropuerto_destino"].unique())
lista_codigo_aeropuerto_origen  = sorted(df["codigo_aeropuerto_origen"].unique())

print(lista_codigo_aeropuerto_destino)
print(f'Número total de aeropuertos de destino: {len(lista_codigo_aeropuerto_destino)}\n')

print(lista_codigo_aeropuerto_origen)
print(f'Número total de aeropuertos de origen: {len(lista_codigo_aeropuerto_origen)}')

In [ ]:
# El valor que aparece de más en codigo_aeropuerto_destino es 'GCK'.
# GCK es el código de aeropuerto para el Aeropuerto de Garden City, ubicado en Garden City, Kansas, Estados Unidos.

codigo_faltante_aeropuerto_destino = set(lista_codigo_aeropuerto_destino) - set(lista_codigo_aeropuerto_origen)
codigo_faltante_aeropuerto_destino

In [ ]:
lista_aeropuerto_origen = sorted(df["aeropuerto_origen"].unique())

print(lista_aeropuerto_origen)
print(f'Aparecen {len(lista_aeropuerto_origen)} aeropuertos.')

In [ ]:
aeropuertos_agrupado = df.groupby(['aeropuerto_origen', 'codigo_aeropuerto_origen']).size().reset_index()

# El código correcto del aeropuerto Tri Cities	es PSC.
codigo_no_coincidente = aeropuertos_agrupado[aeropuertos_agrupado.duplicated(subset=['aeropuerto_origen'], keep=False)]

codigo_no_coincidente

In [32]:
# Asignamos a todos PSC.

df.loc[df['aeropuerto_origen'] == 'Tri Cities', 'codigo_aeropuerto_origen'] = 'PSC'

In [ ]:
# Se realiza correctamente.

df[df['aeropuerto_origen'] == 'Tri Cities']

In [ ]:
# Creamos un diccinoario con los aeropuertos.
diccionario_aeropuertos = dict(zip(aeropuertos_agrupado['codigo_aeropuerto_origen'], aeropuertos_agrupado['aeropuerto_origen']))

# Agregamos el que faltaba.
diccionario_aeropuertos['GCK'] = 'Garden City'

print(diccionario_aeropuertos)

In [35]:
# Creamos una nueva columna con los nombres de los aeropuertos de destino.
df['aeropuerto_destino'] = df['codigo_aeropuerto_destino'].replace(diccionario_aeropuertos)

# A aeropuerto destino también le cambiamos el valor del código para que no tenga dos
df.loc[df['aeropuerto_destino'] == 'Tri Cities', 'codigo_aeropuerto_destino'] = 'PSC'

In [ ]:
df["codigo_aeropuerto_destino"].nunique()

In [ ]:
df["codigo_aeropuerto_origen"].nunique()

AGREGAMOS LAS COLUMNAS `ciudad_destino` y `estado_destino`.

In [38]:
# Utilizamos la columna aeropuerto_origen como referencia para agregar las nuevas columnas.

ciudad_origen_agrupado = df.groupby(['aeropuerto_origen', 'ciudad_origen']).size().reset_index()
estado_origen_agrupado = df.groupby(['aeropuerto_origen', 'estado_origen']).size().reset_index()

In [39]:
# Creamos dos diccionarios donde se relacionan los aeropuertos con las ciudades y los estados.

diccionario_ciudades = dict(zip(ciudad_origen_agrupado['aeropuerto_origen'], ciudad_origen_agrupado['ciudad_origen']))
diccionario_estados = dict(zip(estado_origen_agrupado['aeropuerto_origen'], estado_origen_agrupado['estado_origen']))

In [40]:
# Relacionamos el aeropuerto destino con la ciudad y el estado destino.

df['ciudad_destino'] = df['aeropuerto_destino'].replace(diccionario_ciudades)
df['estado_destino'] = df['aeropuerto_destino'].replace(diccionario_estados)

In [ ]:
# No tenemos datos de salidas de este aeropuerto

aeropuerto_faltante = set(df['aeropuerto_destino']) - set(df['aeropuerto_origen'])

print("Aeropuerto destino del que no salen aviones:", aeropuerto_faltante)

### Ahora vamos a proceder a una búsqueda exhaustiva de las coordenadas (latitud y altitud) y dirección de los aeropuertos de origen y destino, así como las distancias entre los mismos (en millas).

In [ ]:
print(df.columns)

**Lo primero que vamos a hacer para buscar las coordenadas es crear un df con aeropuertos únicos**

In [ ]:
valores_unicos_aeropuertos = df['aeropuerto_destino'].unique()
print(valores_unicos_aeropuertos)

print(f"Número de aeropuertos únicos: {len(valores_unicos_aeropuertos)}")

In [ ]:
df_origen = df[['aeropuerto_origen', 'ciudad_origen', 'estado_origen']].copy()
df_destino = df[['aeropuerto_destino']].copy()
df_destino.columns = ['aeropuerto_origen']

df_destino['ciudad_origen'] = None
df_destino['estado_origen'] = None
df_aeropuertos_concatenados = pd.concat([df_origen, df_destino])

df_aeropuertos_unicos = df_aeropuertos_concatenados.drop_duplicates(subset=['aeropuerto_origen'])
df_aeropuertos_unicos.reset_index(drop=True, inplace=True)


print(df_aeropuertos_unicos)

Buscamos donde está Garden City en Google y le indicamos ciudad y estado


In [ ]:
# Garden City que es el único que aparece como none...
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['aeropuerto_origen'] == "Garden City", 'ciudad_origen'] = "Garden City"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['aeropuerto_origen'] == "Garden City", 'estado_origen'] = "KS"

print(df_aeropuertos_unicos[df_aeropuertos_unicos['aeropuerto_origen'] == "Garden City"])


In [ ]:
# Cambiamos el nombre de las columnas para que sea más cómodo

df_aeropuertos_unicos.rename(columns={
    'aeropuerto_origen': 'nombre_aeropuerto',
    'ciudad_origen': 'ciudad',
    'estado_origen': 'estado'
}, inplace=True)

print(df_aeropuertos_unicos.columns)

## Ahora vamos a utilizar la API de foursquare para buscar las coordenadas.
Como tenemos el nombre del aeropuerto, la ciudad y el estado... vamos a buscar aeropuertos y que nos devuelva latitud y longitud

In [ ]:
# API de Foursquare
CLIENT_ID = "QCDWZLNWZBWTQKPLJLC2GCFIGRICGGQX1D1AJD1JUM0FMUPU"
CLIENT_SECRET = "NVRHLER4BNQ2LHGUEWDFPAIF132KBCT2JHQU5X35MLCS1S0B"
API_KEY = "fsq3gHQLqKS5DaSzw9RzKUqjuW9Ec14h06kJE5xVxKSHAfg="

headers = {"Accept": "application/json", "Authorization": API_KEY}

df_aeropuertos_unicos['latitude'] = None
df_aeropuertos_unicos['longitude'] = None
df_aeropuertos_unicos['direccion'] = None

for index, row in df_aeropuertos_unicos.iterrows():
    url_params = {
        "query": "airport" + row['nombre_aeropuerto'],
        "near": f"{row['ciudad']}, {row['estado']}", 'USA'
        "limit": 1
    }

    
    response = requests.get(url="https://api.foursquare.com/v3/places/search", params=url_params, headers=headers)

    
    if response.status_code == 200:
        data = response.json()
        
        if data['results']:
            result = data['results'][0] 
            latitude = result['geocodes']['main']['latitude']
            longitude = result['geocodes']['main']['longitude']
            direccion = result['location']['formatted_address']
           
            df_aeropuertos_unicos.at[index, 'latitude'] = latitude
            df_aeropuertos_unicos.at[index, 'longitude'] = longitude
            df_aeropuertos_unicos.at[index, 'direccion'] = direccion
    else:
        print(f"Error en la fila {index} con el aeropuerto {row['nombre_aeropuerto']}. Respuesta: {response.status_code}")




print(df_aeropuertos_unicos.head())


In [ ]:
#Es posible que falte algún dato por pruebas anteriores que hemos realizado, por lo tanto ejecutamos una segunda prueba buscando en los datos que no tienen aún

for index, row in df_aeropuertos_unicos.iterrows():    
    if pd.isnull(row['latitude']) or pd.isnull(row['longitude']) or pd.isnull(row['direccion']):        
        url_params['query'] = row['nombre_aeropuerto'] + " airport"

        
        response = requests.get(url="https://api.foursquare.com/v3/places/search", params=url_params, headers=headers)

        
        if response.status_code == 200:
            data = response.json()

            

            if data['results']:
                result = data['results'][0]  
                latitude = result['geocodes']['main']['latitude']
                longitude = result['geocodes']['main']['longitude']
                direccion = result['location']['formatted_address']

                
                df_aeropuertos_unicos.at[index, 'latitude'] = latitude
                df_aeropuertos_unicos.at[index, 'longitude'] = longitude
                df_aeropuertos_unicos.at[index, 'direccion'] = direccion
        else:
            print(f"Error en la segunda búsqueda en la fila {index} con el aeropuerto {row['nombre_aeropuerto']}. Respuesta: {response.status_code}")



print(df_aeropuertos_unicos.head())


In [ ]:
nulos_latitude = df_aeropuertos_unicos['latitude'].isnull().sum()
nulos_longitude = df_aeropuertos_unicos['longitude'].isnull().sum()

print(f"Valores nulos en 'latitude': {nulos_latitude}")
print(f"Valores nulos en 'longitude': {nulos_longitude}")

**Vamos a hacer una comprobación gráfica para ver si a simple vista vemos algún error en las coordenadas... por ejemplo si alguna la ha puesto en medio del mar o en algún país de África por ejemplo...**

In [ ]:
# mapa folium usa
mapa = folium.Map(location=[40, -95], zoom_start=4)

# los marcadores, hay que cambiarlos
for index, row in df_aeropuertos_unicos.iterrows():
    folium.Marker(
        [row['latitude'], row['longitude']], 
        popup=f"{row['nombre_aeropuerto']}", 
        tooltip=row['nombre_aeropuerto']
    ).add_to(mapa)

mapa

**Como podemos ver en el mapa nos situa algún aeropuerto en Ciudad de México o en Tribinidad y Tobago por ejemplo...**
Si que es verdad que hay aeropuertos que tenemos que localizar que no están en territorio de Estados Unidos como puede ser Guam o Samoa Americana... ahora veremos eso más adelante

In [51]:
# Lo tenemos que hacer de forma manual. Algunos eran muy claros... otros hemos investigado un poco a ver si había algo raro

df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Dallas/Fort Worth International', 'latitude'] = 32.897480
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Dallas/Fort Worth International', 'longitude'] = -97.040443

direccion_dallas_fort_worth = "2400 Aviation Dr, DFW Airport, TX 75261, USA"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Dallas/Fort Worth International Airport', 'direccion'] = direccion_dallas_fort_worth

df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Ellison Onizuka Kona International at Keahole', 'latitude'] = 19.738889
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Ellison Onizuka Kona International at Keahole', 'longitude'] = -156.045556
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Ellison Onizuka Kona International at Keahole', 'estado'] = 'HI'

direccion_ellison = "73-200 Kupipi St, Kailua-Kona, HI 96740, USA"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Ellison Onizuka Kona International at Keahole', 'direccion'] = direccion_ellison

df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Guam International', 'latitude'] = 13.48389
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Guam International', 'longitude'] = 144.79722
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Guam International', 'estado'] = 'GU'

direccion_guam = "355 Chalan Pasaheru B224-A, Tamuning, 96913, Guam"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Guam International', 'direccion'] = direccion_guam

df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Francisco C. Ada Saipan International', 'latitude'] = 15.120255
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Francisco C. Ada Saipan International', 'longitude'] = 145.729984
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Francisco C. Ada Saipan International', 'estado'] = 'SAIPAN'

direccion_francisco = "PO Box 501055, Saipan, MP 96950-1055​"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Francisco C. Ada Saipan International', 'direccion'] = direccion_francisco

df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Pago Pago International', 'latitude'] = -14.33166
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Pago Pago International', 'longitude'] = -170.7115031
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Pago Pago International', 'estado'] = 'SAMOA AME'

direccion_samoa = "3 millas al suroeste de Pago Pago, Samoa Americana​"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Pago Pago International', 'direccion'] = direccion_samoa



df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Alexandria International', 'latitude'] = 31.3274
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Alexandria International', 'longitude'] = -92.5498

direccion_alexandria = "1100 Frank Andrews Blvd, Alexandria, LA 71303, USA​"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Alexandria International', 'direccion'] = direccion_alexandria


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'] == 'Alexandria International', 'estado'] = 'LA'


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Bill and Hillary Clinton National|Adams Field', case=False), 'latitude'] = 34.73
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Bill and Hillary Clinton National|Adams Field', case=False), 'longitude'] = -92.22


direccion_bill_hillary_clinton = "1 Airport Drive, Little Rock, AR 72202, USA"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Bill and Hillary Clinton National|Adams Field', case=False), 'direccion'] = direccion_bill_hillary_clinton


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Bill and Hillary Clinton National|Adams Field', case=False), 'estado'] = 'AR'



df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Montgomery Regional', case=False), 'latitude'] = 32.18
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Montgomery Regional', case=False), 'longitude'] = -86.23


direccion_montgomery_regional = "4445 Selma Hwy, Montgomery, AL 36108, USA"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Montgomery Regional', case=False), 'direccion'] = direccion_montgomery_regional


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Montgomery Regional', case=False), 'estado'] = 'AL'



df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Ontario International', case=False), 'latitude'] = 34.0560
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Ontario International', case=False), 'longitude'] = -117.5981


direccion_ontario_international = "Ontario, CA 91761, USA"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Ontario International', case=False), 'direccion'] = direccion_ontario_international


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Ontario International', case=False), 'estado'] = 'CA'



df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Memphis International', case=False), 'latitude'] = 35.0421
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Memphis International', case=False), 'longitude'] = -89.9792


direccion_memphis_international = "2491 Winchester, Suite 113, Memphis, TN 38116"  
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Memphis International', case=False), 'direccion'] = direccion_memphis_international


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Memphis International', case=False), 'estado'] = 'TN'


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Stillwater Regional', case=False), 'latitude'] = 36.1567
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Stillwater Regional', case=False), 'longitude'] = -97.0847


direccion_stillwater_regional = "Stillwater, OK, Estados Unidos"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Stillwater Regional', case=False), 'direccion'] = direccion_stillwater_regional


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Stillwater Regional', case=False), 'estado'] = 'OK'


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Birmingham-Shuttlesworth International', case=False), 'latitude'] = 33.5629
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Birmingham-Shuttlesworth International', case=False), 'longitude'] = -86.7535


direccion_birmingham_shuttlesworth = "5900 Messer Airport Highway, Birmingham, AL 35212"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Birmingham-Shuttlesworth International', case=False), 'direccion'] = direccion_birmingham_shuttlesworth


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Birmingham-Shuttlesworth International', case=False), 'estado'] = 'AL'


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Lovell Field|Chattanooga Metropolitan', case=False), 'latitude'] = 35.035
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Lovell Field|Chattanooga Metropolitan', case=False), 'longitude'] = -85.204


direccion_lovell_field = "1001 Airport Road, Suite 14, Chattanooga, TN 37421"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Lovell Field|Chattanooga Metropolitan', case=False), 'direccion'] = direccion_lovell_field


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Lovell Field|Chattanooga Metropolitan', case=False), 'estado'] = 'TN'


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Lafayette Regional Paul Fournet Field', case=False), 'latitude'] = 30.20520
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Lafayette Regional Paul Fournet Field', case=False), 'longitude'] = -91.98812


direccion_lafayette_regional = "200 Terminal Drive, Suite 200, Lafayette, LA 70508"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Lafayette Regional Paul Fournet Field', case=False), 'direccion'] = direccion_lafayette_regional


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Lafayette Regional Paul Fournet Field', case=False), 'estado'] = 'LA'


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Tri-State/Milton J. Ferguson Field', case=False), 'latitude'] = 38.366944
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Tri-State/Milton J. Ferguson Field', case=False), 'longitude'] = -82.556111


direccion_tri_state_milton = "1449 Airport Road, Huntington, WV 25704"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Tri-State/Milton J. Ferguson Field', case=False), 'direccion'] = direccion_tri_state_milton


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Tri-State/Milton J. Ferguson Field', case=False), 'estado'] = 'WV'


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Gulfport-Biloxi International', case=False), 'latitude'] = 30.408
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Gulfport-Biloxi International', case=False), 'longitude'] = -89.070


direccion_gulfport_biloxi = "14035-L Airport Road, Gulfport, MS 39503"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Gulfport-Biloxi International', case=False), 'direccion'] = direccion_gulfport_biloxi


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Gulfport-Biloxi International', case=False), 'estado'] = 'MS'


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Albert J Ellis', case=False), 'latitude'] = 34.88
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Albert J Ellis', case=False), 'longitude'] = -77.60


direccion_albert_j_ellis = "264 Albert Ellis Airport Road, Richlands, NC 28574"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Albert J Ellis', case=False), 'direccion'] = direccion_albert_j_ellis


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Albert J Ellis', case=False), 'estado'] = 'NC'


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Pensacola International', case=False), 'latitude'] = 30.473333
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Pensacola International', case=False), 'longitude'] = -87.186667


direccion_pensacola_international = "2430 Airport Boulevard, Pensacola, FL 32504"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Pensacola International', case=False), 'direccion'] = direccion_pensacola_international


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Pensacola International', case=False), 'estado'] = 'FL'


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Huntsville International-Carl T Jones Field', case=False), 'latitude'] = 34.63722
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Huntsville International-Carl T Jones Field', case=False), 'longitude'] = -86.77500


direccion_huntsville_international = "1000 Glenn Hearn Blvd SW, Huntsville, AL 35824"
df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Huntsville International-Carl T Jones Field', case=False), 'direccion'] = direccion_huntsville_international


df_aeropuertos_unicos.loc[df_aeropuertos_unicos['nombre_aeropuerto'].str.contains('Huntsville International-Carl T Jones Field', case=False), 'estado'] = 'AL'



## Ahora vamos a hacer un merge para unir el df de aeropuertos únicos con nuestro df principal

In [52]:
# merge para origen
df = pd.merge(
    df,
    df_aeropuertos_unicos[['nombre_aeropuerto', 'latitude', 'longitude', 'direccion']],
    left_on='aeropuerto_origen',
    right_on='nombre_aeropuerto',
    how='left',
    suffixes=(None, '_origen')  # esto es para evita la creación de columnas _x
)



if 'nombre_aeropuerto_origen' in df.columns:
    df.drop('nombre_aeropuerto_origen', axis=1, inplace=True)

# merge para destino
df = pd.merge(
    df,
    df_aeropuertos_unicos[['nombre_aeropuerto', 'latitude', 'longitude', 'direccion']],
    left_on='aeropuerto_destino',
    right_on='nombre_aeropuerto',
    how='left',
    suffixes=(None, '_destino')  # esto igual que lo de antes para evitar la creación de columnas _y
)



if 'nombre_aeropuerto_destino' in df.columns:
    df.drop('nombre_aeropuerto_destino', axis=1, inplace=True)

In [53]:
#df.drop('nombre_aeropuerto', axis=1, inplace=True)


# ordenar las columnas para tenerlas mejor visualmente para buscar cositas
nuevo_orden = ['aerolinea', 'fecha', 'numero_vuelo', 'numero_cola',
               'hora_salida_programada', 
               'hora_salida_real', 'duracion_programada_vuelo', 'duracion_real',
               'retraso_salida', 'hora_despegue', 'tiempo_pista_salida',
               'tiempo_retraso_aerolinea', 'tiempo_retraso_clima',
               'tiempo_retraso_sistema_aviacion', 'tiempo_retraso_seguridad',
               'retraso_llegada', 'ciudad_origen', 'estado_origen',
               'aeropuerto_origen', 'codigo_aeropuerto_origen', 'direccion', 'latitude', 'longitude', 
               'hora_llegada_real', 'dia_semana', 'anio', 'mes', 'fin_de_semana', 'festivos',
               'ciudad_destino', 'estado_destino',
               'aeropuerto_destino', 'codigo_aeropuerto_destino', 'direccion_destino',
               'latitude_destino', 'longitude_destino'
               ]



df = df[nuevo_orden]

In [ ]:
df.columns

In [55]:
df.rename(columns={
    'latitude': 'latitude_origen',
    'longitude': 'longitude_origen',
    'direccion': 'direccion_origen'
}, inplace=True)

**Ahora vamos a calcular las millas de distancia entre aeropuertos**

In [56]:
# Definir la función que implementa la fórmula de Haversine
def haversine(latitude_origen, longitude_origen, latitude_destino, longitude_destino):
    # Convertir latitudes y longitudes de grados a radianes
    lat1, lon1, lat2, lon2 = map(np.radians, [latitude_origen, longitude_origen, latitude_destino, longitude_destino])
    
    # Diferencia de latitudes y longitudes
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Aplicar la fórmula de Haversine
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radio de la Tierra en millas
    R = 3958.8
    distance = R * c
    return distance

# Aplicar la función al DataFrame para calcular la distancia y crear una nueva columna
df['distancia_millas'] = df.apply(lambda row: haversine(row['latitude_origen'], row['longitude_origen'], row['latitude_destino'], row['longitude_destino']), axis=1)


In [ ]:
df

In [ ]:
hay_nan = df.isna().any().any()

if hay_nan:
    print("Hay NaN en el dataset.")
else:
    print("No hay NaN en el dataset.")

In [ ]:
df.isna().sum().sum()

En este punto no tenemos variables sin tratar, todas se encuentran en formatos adecuados para su estudio y tenemos todas las filas completas sin Nan's.

#### Finalmente guardamos los datos en un archivo *pickle* para utilizarlo en el **EDA** y en **Machine Learning posteriormente**.

In [62]:
df.to_pickle('data/pickle/vuelos_limpio.pkl')